# Analysis Utils

In [17]:
def return_oov_tokens(cleaned_reviews):
    """Returns a set of out-of-vocabulary tokens.
       Use it to check which words in our review content
       are unrecognisable to SpaCy.
    """
    oov_tokens = set()
    
    for review in cleaned_reviews:
        cleaned_reviews_doc = loaded_lang_model(review)

        for token in cleaned_reviews_doc:
            if token.is_oov and (not token.is_space):
                oov_tokens.add(token.text)
    return oov_tokens

# Running Utils

In [8]:
import spacy
import pandas as pd
import os

def remove_punctuation(text):
    """Removes punctuation."""
    acceptable = ['£','%']
    text = text.replace('\n', " ").replace('\t','').replace('\r','')
    clean_chars = [char.lower() for char in text if char.isalnum() or char.isspace() or (char in acceptable)]
    return "".join(clean_chars)



def all_files_in_folder(current_folder_path):
    """Gets all names of the files in the 'current' folder."""
    files_list = []
    files_n_dirs = os.listdir(current_folder_path) # folders and files in current folder

    for file in files_n_dirs:
        path_to_file = os.path.join(current_folder_path,file) # folder abs path + file_name
        is_file = os.path.isfile(path_to_file)
        if is_file == True:
            files_list.append(path_to_file)
    return files_list



def clean_all_files(files_list):
    """Returns a list of all reviews without punctuation."""
    reviews = []

    for file in files_list:
        df = pd.read_csv(file)
        corpus_df = df['content']
        corpus_df = corpus_df.apply(remove_punctuation) # series object
        reviews.append(corpus_df.tolist())
    return reviews




def extend_review_list(cleaned_reviews):
    """Returns a list with all the elements of all the lists in the cleaned_revis list."""
    extended_list = []
    for item_list in cleaned_reviews:
        extended_list.extend(item_list)
    return extended_list




def lemmatise_and_rmv_stopwords(review):
    """Returns lemmatised review content without stopwords."""
    no_stop_wrds_review = " ".join([token.lemma_ for token in review if token.is_stop == False])
    return no_stop_wrds_review

# Main body

In [9]:
folder = '/home/mrfox/Desktop/bt_group_project/data'
list_of_review_files = all_files_in_folder(folder)
cleaned_reviews = extend_review_list(clean_all_files(list_of_review_files))
print(cleaned_reviews)
#DATA PROFILING:
# list of 80 lists, each is one page worth of reviews
# there are almost exactly 40 reviews on each page, incl. the last one
# 80*40 = 3200, we are 8 reviews short of that number, this is probably
# due to 8 people not submitting a review, just the ratings


loaded_lang_model = spacy.load("en_core_web_lg") # use lg to be able to use word vectors


['my broadband has been down for 12 days now with no resolve i have contacted support constantly and even had open reach say they visited me lie and i would repair it myself so i was put back in urgency list aaaagghhh', 'what an absolute shambles bt and kelly communications are i orderd fast fiber in mid september and kellys who were supposed to install the wire didnt bother so that was a major delay bt staff not botherd and kept blaming everyone else its now the begining of december and im still waiting to be connected steer well clear of bt and its associates shocking display on how to run a company and would not reccomend them to anyone even my worst enemy', 'just read some reviews and realised that the worst service ive ever experienced is normal for bt  how can this company be so poor at delivering what they promise  organise  im going to email the ceo to see if they are interested or aware of how poor the bt company has become', 'worst internet i ever had in my 22 year life i hav

In [16]:
oov_tokens = return_oov_tokens(cleaned_reviews)
print(oov_tokens)

{'britih', 'resultswas', 'speedtv', 'servicenumber', 'portsnot', 'autoemails', '5megs', '6349', '30pm', 'emaili', 'faultnext', '0310', 'cablesaverage', 'c£49', 'spees', '20mbafter', 'uploadi', 'moble2nd', 'everythingand', 'annoyingits', 'companyfirst', '100£128', 'arewe', 'detachedi', 'worsed', '11more', 'isyou', 'costsyou', '4618', 'arnside', 'tradferred', '2lol', 'rewardsgoing', 'teamsproblem', 'customerslook', 'forconvenient', '121216', 'exchane', 'problemsprovided', '5361', 'mbops', 'cancellingphone', 'sportafter', 'waitingno', 'hadthe', '23rdi', 'serviceyou', 'curedi', 'hsow', 'promisedorder', 'btyou', 'appallingleaving', 'btdo', '20725', 'broadbandfast', 'excustomer', 'btwifiwithfon', 'problemswhat', '65mbs', 'broadbandi', 'tondeoaetment', 'themwill', '19mbps', 'materialisetrying', '81pmi', 'moneyfor', '7595', 'uselessif', 'ringcannot', 'weeksthey', 'lineso', 'connectionoften', '80100', 'quaky', 'calledbut', 'copperwire', 'cancellationsi', 'manymany', 'numoures', 'skavar', 'enoug